In [52]:
# from transformers import T5Tokenizer, T5ForConditionalGeneration

# tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
# model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

# input_text = "translate English to German: How old are you?"
# input_ids = tokenizer(input_text, return_tensors="pt").input_ids

# outputs = model.generate(input_ids)
# print(tokenizer.decode(outputs[0]))

# Load Model

In [97]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model = AutoModelForSeq2SeqLM.from_pretrained("shorthillsai/flan-t5-large-absa", device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("shorthillsai/flan-t5-large-absa", truncation=True)

prompt = """Find the aspect based sentiment for the given review. 'Not present' if the aspect is absent.\n\nReview:I love the screen of this laptop and the battery life is amazing.\n\nAspect:Battery Life\n\nSentiment: """

input_ids = tokenizer(prompt, return_tensors="pt").to("cuda").input_ids
instruct_model_outputs = model.generate(input_ids=input_ids)
instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)


c:\Users\Nimra\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\generation\utils.py:1353: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


# Load Dataset

In [98]:
import os
import pandas as pd
path = "flan_t5/results/"
data_path="MAMS/"
filename="ACSA_test"
if(os.path.isfile(path+data_path+filename+"_t5.csv")):
    res = pd.read_csv(path+data_path+filename+"_t5.csv")
else:
    res = pd.DataFrame(columns=['dataset_name', 'sentence', 'aspect', 'polarity', 'sentiment_t5'])
print(len(res))
res.head(3)

901


,dataset_name,sentence,aspect,polarity,sentiment_t5
0,MAMS/ACSA_test,"We went again and sat at the bar this time, I ...",place,neutral,Negative
1,MAMS/ACSA_test,"We went again and sat at the bar this time, I ...",food,negative,Negative
2,MAMS/ACSA_test,"The food was good, but it's not worth the wait...",food,positive,Neutral


In [99]:
import pandas as pd

df = pd.read_csv('processed_'+data_path+filename+'_single.csv')
# df = pd.read_csv('processed_'+data_path+filename+'_single.csv')
print(len(df))
df.head(2)

901


,sentence,aspect_category,polarity
0,"We went again and sat at the bar this time, I ...",place,neutral
1,"We went again and sat at the bar this time, I ...",food,negative


# Iterate each Row

In [100]:
df.dropna(subset=['aspect_category'], inplace=True)
print(len(df), len(res))
# for sentence, aspec, senti in zip(df['sentence'][len(res):1],df['aspect_term'][len(res):1], df['polarity'][len(res):1]):
for sentence, aspec, senti in zip(df['sentence'][len(res):],df['aspect_category'][len(res):], df['polarity'][len(res):]):
    temp = pd.DataFrame(columns=['dataset_name', 'sentence', 'aspect', 'polarity', 'sentiment_t5'])   
    prompt = f"""Review: {sentence}
    Aspect List: {aspec.replace('#', ' ').lower()}
    Predict the sentiment for mentioned aspect in the review. Assign one of the following sentiment to the aspect: Positive , Negative , Neutral.
    Please provide the sentiment only. No any explanation or extra text"""
    # print(prompt)
    input_ids = tokenizer(prompt, return_tensors="pt").to("cuda").input_ids
    instruct_model_outputs = model.generate(input_ids=input_ids)
    instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

    data_to_append = {
    'dataset_name': data_path+filename,
    'sentence': sentence,
    'aspect': aspec,
    'polarity': senti,
    'sentiment_t5': instruct_model_text_output 
    }

    print(data_to_append)
    temp = temp.append(data_to_append, ignore_index=True)
    file_path = path+data_path+filename+"_t5.csv"
    if os.path.exists(file_path):   
        temp.to_csv(file_path, index=False, mode='a', header=False)
    else:
        temp.to_csv(file_path, index=False, mode='a')
    

901 901


In [38]:
# #Old Prompt
# for sentence, aspec, senti in zip(df['sentence'][len(res):1],df['aspect_category'][len(res):1], df['polarity'][len(res):1]):
# # for sentence, aspec, senti in zip(df['sentence'][len(res):],df['aspect_category'][len(res):], df['polarity'][len(res):]):
#     temp = pd.DataFrame(columns=['dataset_name', 'sentence', 'aspect', 'polarity', 'sentiment_t5'])
#     prompt = f"""Find the aspect based sentiment for the given review. 'Not present' if the aspect is absent.
#     \n\nReview: {sentence.lower()}.
#     \n\nAspect: {aspec.replace('#', ' ').lower()} \n\nSentiment: """
#     # print(prompt)
#     input_ids = tokenizer(prompt, return_tensors="pt").to("cuda").input_ids
#     instruct_model_outputs = model.generate(input_ids=input_ids)
#     instruct_model_text_output = tokenizer.decode(instruct_model_outputs[0], skip_special_tokens=True)

#     data_to_append = {
#     'dataset_name': data_path+filename,
#     'sentence': sentence,
#     'aspect': aspec,
#     'polarity': senti,
#     'sentiment_t5': instruct_model_text_output 
#     }

#     print(data_to_append)
#     temp = temp.append(data_to_append, ignore_index=True)
#     temp.to_csv(path+data_path+filename+"_category_t5.csv", index=False, mode='a')

In [38]:
%pwd

'd:\\ABSA work\\codes'